In [1]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from PIL import Image
import pytesseract
import numpy as np
import requests
from io import BytesIO

# Dataset

To obtain the daataset we have done web scrapping in the https://www.poetryfoundation.org/ web. First we obtain the number of poems of a given period and then we iterate over all the pages to obtain the title, author and the poem itself.

In [3]:
def extract_total_results(url):
    response = requests.get(url)
    data = response.json()
    total_results = data['TotalResults']
    return total_results

In [4]:
def extract_text_from_image(url):
  response = requests.get(url)
  img_bytes = BytesIO(response.content)


  img = np.array(Image.open(img_bytes))

  img_np = np.array(img)
  
  if img_np.size > 0:
      text = pytesseract.image_to_string(img_np)
      text = text.replace('\n', ' ')
  else:
      text = None

  return text

In [5]:
url = 'https://www.poetryfoundation.org/poetrymagazine/browse?contentId=26576'

response = requests.get(url)
poem_soup=BeautifulSoup(response.text, 'html.parser')
poem=poem_soup.find(class_='c-assetStack-media')
img = poem.find('img')
src_value = img['src']
print(src_value)
text=extract_text_from_image(src_value)

print(text)

https://static.poetryfoundation.org/jstor/i20585549/pages/14.png
POETRY  three sonnets of ronsard  COMME ON VOIT SUR LA BRANCHE  As one sees on its branch, in May, the rose  In its bright youth and new astonishment,  Making the heavens jealous of its tint  When, touched with dew, inthe first light it glows, Then, in its petal, grace and love repose,  Filling the gardens and the trees with scent;  But, scourged by the sun’s heat, by the rain bent, Drooping, it dies: petal from petal blows.  So your bright youth, which earth and heaven adore, Lies now in ashes, for no love can strive  Against the murdering Fate. For obsequy,  Receive my tears, this vase of milk I pour,  This basket full of flowers, so that alive  And dead, your body shall all roses be.  200 


In [6]:
def extract_poems(url,period):
    response = requests.get(url)
    data = response.json()

    # Extract attributes from each entry
    entries = []
    for entry in data['Entries']:
        entry_data = {
            'id': entry['id'],
            'title': entry['title'],
            'author': entry['author'],
            'snippet': entry['snippet'],
            'link': entry['link'],
            'categories': [category['title'] for category in entry['categories']],
            'period': period
        }

        # Fetch the poem content from the link
        poem_response = requests.get(entry_data['link'])
        poem_soup = BeautifulSoup(poem_response.text, 'html.parser')
        poem = poem_soup.find(class_='o-poem')
        if poem != None:
          poem_text = poem.get_text(separator=' ', strip=True) # If we wanted to take into accouent the form of the poem use \n as separator

          # Add poem text to the entry data
          entry_data['poem'] = poem_text
        
        else: # When the poem is an image
          poem=poem_soup.find(class_='c-assetStack-media')

          if poem == None: #  It is audio
              entry_data['poem'] = None
          else:
            img = poem.find('img')
            src_value = img['src']
            poem_text = extract_text_from_image(src_value)
            entry_data['poem'] = poem_text

        entries.append(entry_data)

    # Create a pandas DataFrame
    df = pd.DataFrame(entries)
    return df

In [7]:
# Initialize an empty list to store individual DataFrames
dfs = []
period_ids = {
    'Middle English': 158,
    'Augustan': 149,
    'Renaissance': 163,
    'Romantic': 164,
    'Victorian': 165,
    'Fugitive': 153,
    'Georgian': 154,
    'Harlem Renaissance': 155,
    'Imagist': 156,
    'Modern': 159,
    'Objectivist': 162,
    'Beat': 150,
    'Black Arts Movement': 304,
    'Black Mountain': 151,
    'Confessional': 152,
    'Language Poetry': 157,
    'New York School': 160,
    'New York School (2nd Generation)': 161
}

In [8]:
#Loop over all the periods to find how many entries they have
for i in period_ids:
  total_results = extract_total_results("https://www.poetryfoundation.org/ajax/poems?page=1&sort_by=title&school-period={}".format(period_ids[i]))
  print("Extracting poems of ", i)
  # Loop through the URLs
  for j in range(total_results // 20 + 1): # 20 poems per page
      url = "https://www.poetryfoundation.org/ajax/poems?page={}&sort_by=title&school-period={}".format(j+1, period_ids[i])
      # Extract DataFrame from each URL and append to the list
      dfs.append(extract_poems(url,i))



Extracting poems of  Middle English
Extracting poems of  Augustan
Extracting poems of  Renaissance
Extracting poems of  Romantic
Extracting poems of  Victorian
Extracting poems of  Fugitive
Extracting poems of  Georgian
Extracting poems of  Harlem Renaissance
Extracting poems of  Imagist


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7efc95db2b60>

In [9]:
# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
print(len(final_df))
final_df.head()

final_df.to_csv("poems_data.csv", index=False, quoting=1)

2318


### Full Poems Extraction

In [10]:
def extract_poems(url):
    response = requests.get(url)
    data = response.json()

    # Extract attributes from each entry
    entries = []
    for entry in data['Entries']:
        entry_data = {
            'id': entry['id'],
            'title': entry['title'],
            'author': entry['author'],
            'snippet': entry['snippet'],
            'link': entry['link'],
            'categories': [category['title'] for category in entry['categories']]            
        }

        # Fetch the poem content from the link
        poem_response = requests.get(entry_data['link'])
        poem_soup = BeautifulSoup(poem_response.text, 'html.parser')
        poem = poem_soup.find(class_='o-poem')
        if poem != None:
          poem_text = poem.get_text(separator=' ', strip=True) # If we wanted to take into accouent the form of the poem use \n as separator

          # Add poem text to the entry data
          entry_data['poem'] = poem_text
        else: # When the poem is an image
          poem=poem_soup.find(class_='c-assetStack-media')
          
          if poem == None: ## It is audio
              entry_data['poem'] = None
          else:
            img = poem.find('img')
            src_value = img['src']
            poem_text = extract_text_from_image(src_value)
            entry_data['poem'] = poem_text

        entries.append(entry_data)

    # Create a pandas DataFrame
    df = pd.DataFrame(entries)
    return df




In [15]:
total_results = extract_total_results("https://www.poetryfoundation.org/ajax/poems?page=1")
dfs = []
n_poems = 0
count =0

for j in range(73,total_results // 20 + 1): # 20 poems per page
  

    url = "https://www.poetryfoundation.org/ajax/poems?page={}".format(j+1)
    # Extract DataFrame from each URL and append to the list
    dfs.append(extract_poems(url))

    n_poems += 20
    count += 1
    if count == 10:
        print("We have counted already ", n_poems)
        count = 0



We have counted already  200
We have counted already  400
We have counted already  600
We have counted already  800
We have counted already  1000
We have counted already  1200
We have counted already  1400
We have counted already  1600
We have counted already  1800
We have counted already  2000
We have counted already  2200
We have counted already  2400
We have counted already  2600
We have counted already  2800


KeyboardInterrupt: 

In [16]:
# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
print(len(df))
df.head()

df.to_csv("full_poems_data.csv", index=False, quoting=1)

2880
